In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from sklearn.model_selection import KFold

# Seed

In [12]:
seed_number = 486

torch.manual_seed(seed_number)
torch.cuda.manual_seed(seed_number)
torch.cuda.manual_seed_all(seed_number) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
#np.random.seed(seed_number)
#random.seed(seed_number)

# Data

In [13]:
cloud_ld=
cloud_ld_test=

SyntaxError: invalid syntax (307590590.py, line 1)

In [14]:
X      = cloud_ld
X_test = cloud_ld_test

X      = torch.tensor(X, dtype=torch.uint8)
X_test = torch.tensor(X_test, dtype=torch.uint8)

print(X.shape)
print(X_test.shape)

NameError: name 'cloud_ld' is not defined

In [ ]:
y      = cloud_ld.targets
y_test = cloud_ld_test.targets

y      = torch.tensor(y, dtype=torch.uint8)
y_test = torch.tensor(y_test, dtype=torch.uint8)

print(y.shape)
print(y_test.shape)

# 데이터 크기, 형태 확인후 넣을지말지 결정

In [ ]:
X = X.permute(0,3,1,2)
X_test = X_test.permute(0,3,1,2)

print(X.shape)
print(X_test.shape)

# Nomalization

In [ ]:
X = X / 255.
X_test  = X_test / 255.

print(X.shape)
print(X_test.shape)

# Sampling

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=seed_number)

print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

NameError: name 'X' is not defined

# CPU possiblity

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

X_train = X_train.to(device=device)
X_valid = X_valid.to(device=device)
X_test  = X_test.to(device=device)

y_train = y_train.to(device=device)
y_valid = y_valid.to(device=device)
y_test  = y_test.to(device=device)

# Batch size (이후 수정)

In [ ]:
train_ds = torch.utils.data.TensorDataset(X_train, y_train)
valid_ds = torch.utils.data.TensorDataset(X_valid, y_valid)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_ds, batch_size=32, shuffle=False)

# Model (layer1 size 수정 필요)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv_1 = torch.nn.Conv2d(1, 6, 5)
        self.conv_2 = torch.nn.Conv2d(6, 16, 5)
        self.pool   = torch.nn.MaxPool2d(2, 2)
        
        self.fully_connected_layer_1 = torch.nn.Linear(16*5*5, 120)
        self.fully_connected_layer_2 = torch.nn.Linear(120, 84)
        self.fully_connected_layer_3 = torch.nn.Linear(84,  10)
        
    def forward(self, xb):
        xb = xb.unsqueeze(1)
        
        out = self.conv_1(xb)
        out = torch.nn.ReLU()(out)
        out = self.pool(out)
        
        out = self.conv_2(out)
        out = torch.nn.ReLU()(out)
        out = self.pool(out)
        
        out = torch.flatten(out, 1)
        
        out = self.fully_connected_layer_1(out)
        out = torch.nn.ReLU()(out)
        out = self.fully_connected_layer_2(out)
        out = torch.nn.ReLU()(out)
        out = self.fully_connected_layer_3(out)
        
        return out

In [ ]:
model = Net().to(device)
print(model)

In [ ]:
from sklearn.model_selection import KFold

# Cost loss

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

# Optimizer

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
import time 

for epoch in range(3):
    start = time.time()
    train_loss, train_count = 0., 0.
    valid_loss_history = []
    train_loss_history = []
    is_train=True
    
    for xb, yb in train_loader:
        prediction = model(xb)
        loss = criterion(prediction, yb)
        train_loss += loss.item()*len(yb)
        train_count += len(yb)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
        
    with torch.no_grad():
        valid_loss = 0
        valid_accuracy, valid_count = 0., 0.
        
        for xb, yb in valid_loader:
            prediction = model(xb)
            valid_loss += criterion(prediction, yb)*len(yb)
            valid_accuracy += prediction.data.max(1)[1].eq(yb.data).sum()
            valid_count += len(yb)
        
    train_loss /= train_count
    valid_loss /= valid_count    
    valid_accuracy /= valid_count
    
    train_loss_history.append(train_loss)
    valid_loss_history.append(valid_loss)
    
    if min(valid_loss_history) == valid_loss:
        early_stopping = 0
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': valid_loss,
        }, './saved_model.pt')
        
    if early_stopping == 3:
        is_train = False
            
    print(f"======== Epoch {epoch+1} =======")
    print(f"Loss => train:{train_loss:.5f}, valid:{valid_loss:.5f}")
    print(f"Accuracy => {valid_accuracy*100:.2f}%, Elapsed time => {time.time()-start:.3f} sec")
    print("=======================================\n")